In [1]:
import numpy as np
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [2]:
nx = 101
ny = 101
xmin = 0
xmax = 1.0
ymin = -0.5
ymax = 0.5

Lx = xmax-xmin
Ly = ymax-ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

x = np.linspace(xmin, xmax, num = nx)
y = np.linspace(ymin, ymax, num = ny)
X, Y = np.meshgrid(x, y)

b = (-2 * (np.pi / 2)**2 
     * np.sin(np.pi * X / Lx) 
     * np.cos(np.pi * Y / Ly))


p0 = np.zeros((ny, nx))
pe = poisson_solution(x, y, Lx, Ly)

In [3]:
def poisson_2d_steepest_descent(p0, b, dx, dy, 
                                maxiter = 20000, rtol = 1e-6):
    def A(p):
        # compute the laplacian of p
        return((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
              (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    
    p = p0.copy()
    diff = rtol + 1
    ite = 0
    rk = np.zeros_like(p)
    Ar = np.zeros_like(p)
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # compute residual rk.
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        # compute step size alpha
        Ar[1:-1, 1:-1] = A(rk) 
        alpha = np.sum(rk * rk) / np.sum(rk * Ar)
        # update solution
        p = pk + alpha * rk
        # compute relative difference
        diff = l2_norm(p, pk)
        # increment iteration index
        ite = ite + 1
    return p, ite, diff

In [4]:
p, ites, diff = poisson_2d_steepest_descent(p0, b, dx, dy, rtol = 1e-10)
print(ites, diff)

2 1.2374874917997386e-16


In [12]:
b = (np.sin(np.pi * X / Lx) 
     * np.cos(np.pi * Y / Ly)
    +  np.sin(6 * np.pi * X / Lx) 
     * np.cos(6 * np.pi * Y / Ly))

In [13]:
def poisson_2d_cojugate_gradients(p0, b, dx, dy, 
                                maxiter = 20000, rtol = 1e-6):
    def A(p):
        # compute the laplacian of p
        return((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
              (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    
    p = p0.copy()
    diff = rtol + 1
    ite = 0
    r = np.zeros_like(p)
    Ad = np.zeros_like(p)
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        #
        Ad[1:-1, 1:-1] = A(d) 
        alpha = np.sum(r * r) / np.sum(d * Ad)
        # 
        p = pk + alpha * d        
        # 
        r = rk - alpha * Ad
        #
        beta = np.sum(r * r) / np.sum(rk * rk)
        d = r + beta * d
        # 
        diff = l2_norm(p, pk)
        # 
        ite = ite + 1
    return p, ite, diff

In [14]:
p, ites, diff = poisson_2d_cojugate_gradients(p0, b, dx, dy, rtol = 1e-10)
print(ites, diff)

72 3.438765648895609e-11
